# Convert Multiple Jupyter Notebooks to Anki Cards CSV
This notebook reads multiple Jupyter Notebook files from a directory, extracts the content, and generates corresponding CSV files for Anki cards.

In [ ]:
import nbformat
import pandas as pd
import os

### Define function to process a single Jupyter Notebook file

In [ ]:
def process_notebook(notebook_path):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    # Extract content
    fronts = []
    backs = []

    # Initialize flags and text containers
    in_intro = False
    intro_text = ""
    code_text = ""

    for cell in nb.cells:
        if cell.cell_type == 'markdown':
            if not in_intro:
                intro_text = cell.source
                in_intro = True
            else:
                # Append front and back content
                fronts.append(intro_text)
                backs.append(code_text)
                intro_text = cell.source
                code_text = ""
        elif cell.cell_type == 'code':
            code_text += cell.source + '\n'
            if 'outputs' in cell:
                for output in cell.outputs:
                    if 'text' in output:
                        code_text += ''.join(output['text']) + '\n'

    # Append the last set of front and back content
    if intro_text and code_text:
        fronts.append(intro_text)
        backs.append(code_text)

    # Create DataFrame
    df = pd.DataFrame({
        'Front': fronts,
        'Back': backs
    })

    # Generate CSV file
    csv_path = os.path.splitext(notebook_path)[0] + '_anki_cards.csv'
    df.to_csv(csv_path, index=False)

    print(f'Anki cards CSV file saved to: {csv_path}')

### Process all Jupyter Notebook files in a directory

In [ ]:
# Specify the directory containing Jupyter Notebooks
directory_path = 'path/to/your/notebooks'  # Replace with your directory path

# Iterate through all .ipynb files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.ipynb'):
        notebook_path = os.path.join(directory_path, filename)
        process_notebook(notebook_path)